In [1]:
import mlflow
import pandas as pd
from mlflow import tracking
from mlflow.entities import ViewType
import yaml
import json
import numpy as np
import sys  # Import the sys module for stderr
import os
import time

In [2]:
# Define a custom function to convert NumPy objects to standard Python objects
def numpy_to_python(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, np.generic):
        return np.asscalar(obj)
    return obj

# Set the MLflow tracking server URI
#tracking_uri = "http://127.0.0.1:5000"  # Replace with your MLflow server URI
tracking_uri = os.environ["MLFLOW_TRACKING_URI"]  # Replace with your MLflow server URI
print(f"connecting with: {tracking_uri}", file=sys.stderr)
tracking.set_tracking_uri(tracking_uri)

connecting with: http://server:5000


In [3]:
# Retrieve all experiments and create a dictionary to map experiment IDs to names
print(f"searching for experiments", file=sys.stderr)
experiment_id_to_name = {}
for exp in mlflow.search_experiments():
    if "_test" not in exp.name:
        print(f" - found {exp.name}", file=sys.stderr)
        experiment_id_to_name[exp.experiment_id] = exp.name

# Retrieve all experiments
all_experiments = list(experiment_id_to_name.keys())

searching for experiments
 - found unconditional-hybrid-pre-trained-circles
 - found unconditional-hybrid-pre-trained-moons
 - found unconditional-hybrid-circles
 - found unconditional-hybrid-moons
 - found conditional-circles
 - found conditional-moons
 - found unconditional-circles
 - found unconditional-moons
 - found unconditional-moons


In [4]:
#list(experiment_id_to_name.items())[12]

In [5]:
ts = time.time()
print("downloading runs", file=sys.stderr)
all_runs = []
progress_counter = 0
for experiment_id in all_experiments:
    try:
        runs = mlflow.search_runs(
            experiment_ids=[experiment_id],
            run_view_type=ViewType.ALL,
            max_results=1000000
        )
        all_runs.append(runs)
        progress_counter += 1
        print(f"Downloaded {progress_counter}/{len(all_experiments)} experiments", file=sys.stderr)
        time.sleep(1) # to avoid to many requests error (and speed up download time??)
    except Exception as e:
        all_runs.append(None)
        progress_counter += 1
        print(f"skipping {progress_counter} {list(experiment_id_to_name.items())[progress_counter-1]}", file=sys.stderr)
        
te = time.time()
te-ts

downloading runs
Downloaded 1/9 experiments
Downloaded 2/9 experiments
Downloaded 3/9 experiments
Downloaded 4/9 experiments
Downloaded 5/9 experiments
Downloaded 6/9 experiments
Downloaded 7/9 experiments
Downloaded 8/9 experiments
Downloaded 9/9 experiments


89.35411238670349

In [6]:
#all_runs[15]
#experiment_id_to_name[all_experiments[14]]

In [7]:
df = pd.concat(all_runs)
df["experiment_name"] = df.apply(lambda row: experiment_id_to_name[row["experiment_id"]], axis=1)

df = df.dropna(axis=1, how='all')
df = df.sort_values(["metrics.val_loss"])

for c in df.columns:
    print(c)

run_id
experiment_id
status
artifact_uri
start_time
end_time
metrics.min_loss
metrics.val_loss
metrics.final_epoch
metrics.loss
metrics.min_val_loss
metrics.best_epoch
metrics.lr
params.model_kwds.freeze_base_model
params.opt_beta_2
params.model_kwds.base_parameter_kwds.conditional
params.validation_split
params.run_name
params.dataset_kwds.noise
params.dataset_kwds.scale
params.max_queue_size
params.opt_amsgrad
params.model_kwds.base_distribution_kwds.smooth_bounds
params.use_multiprocessing
params.fit_kwds.reduce_lr_on_plateau
params.batch_size
params.epochs
params.class_weight
params.fit_kwds.epochs
params.opt_epsilon
params.workers
params.model_kwds.parameter_kwds.activation
params.shuffle
params.fit_kwds.learning_rate
params.model_kwds.parameter_kwds.hidden_units
params.monitor
params.patience
params.dataset_kwds.dataset_name
params.steps_per_epoch
params.results_path
params.baseline
params.fit_kwds.batch_size
params.model_kwds.distribution
params.model_kwds.distribution_kwds.orde

In [8]:
# clean run names for easy aggregation and comparison later on
def remove_exp_from_run_name(row):
    exp_name = row["experiment_name"]
    words = exp_name.split("-")

    # Remove the last word by slicing the list of words
    words_without_last = words[:-1]

    # Join the remaining words back into a single string using "-" as the delimiter
    exp_name = "_".join(words_without_last)
    
    run_name = row["tags.mlflow.runName"]
    new_name = run_name.replace(f"{exp_name}_", "")
    return new_name

df["tags.mlflow.runName"] = df.apply(remove_exp_from_run_name, axis=1)

## Generate summarizing dataframe

In [9]:
grouped_df  = df.groupby(["experiment_name", "tags.mlflow.runName"])

# dipslay
grouped_df.apply(lambda r: r[:])

run_id  \
experiment_name     tags.mlflow.runName                                         
conditional-circles bernstein_flow      376  03f38a5a4a784c6ea2381a134d5301d9   
                                        373  7fda6820ed6f4a97bab468c2cd397049   
                                        354  4610459880544b7993a0749702c40623   
                                        352  8c25ee6e07f24fc08f42e051d44da21f   
                                        360  01f3f008570149bdaa417363726937e2   
...                                                                       ...   
unconditional-moons multivariate_normal 14   f71f86577d7b49469af336f151a59703   
                                        17   40a75642a3f6459591daa3fb0a1c28d3   
                                        11   44662cbea86a4705911492f74ef7631a   
                                        16   3d3168c5baf14f0e9a10b32e6594eb84   
                                        12   ab87a04ba6cf4c58858c2acde5c5c83b   

                                                  experiment_id    status  \
experiment_name     tags.mlflow.runName                                     
conditional-circles bernstein_flow      376  788739528573814156  FINISHED   
                                        373  788739528573814156  FINISHED   
                                        354  788739528573814156  FINISHED   
                                        352  788739528573814156  FINISHED   
                                        360  788739528573814156  FINISHED   
...                                                         ...       ...   
unconditional-moons multivariate_normal 14   266110308394071622  FINISHED   
                                        17   266110308394071622  FINISHED   
                                        11   266110308394071622  FINISHED   
                                        16   266110308394071622  FINISHED   
                                        12   266110308394071622  FINISHED   

                                                                                  artifact_uri  \
experiment_name     tags.mlflow.runName                                                          
conditional-circles bernstein_flow      376  file:///mlruns/788739528573814156/03f38a5a4a78...   
                                        373  file:///mlruns/788739528573814156/7fda6820ed6f...   
                                        354  file:///mlruns/788739528573814156/461045988054...   
                                        352  file:///mlruns/788739528573814156/8c25ee6e07f2...   
                                        360  file:///mlruns/788739528573814156/01f3f0085701...   
...                                                                                        ...   
unconditional-moons multivariate_normal 14   file:///mlruns/266110308394071622/f71f86577d7b...   
                                        17   file:///mlruns/266110308394071622/40a75642a3f6...   
                                        11   file:///mlruns/266110308394071622/44662cbea86a...   
                                        16   file:///mlruns/266110308394071622/3d3168c5baf1...   
                                        12   file:///mlruns/266110308394071622/ab87a04ba6cf...   

                                                                  start_time  \
experiment_name     tags.mlflow.runName                                        
conditional-circles bernstein_flow      376 2023-09-21 05:41:02.927000+00:00   
                                        373 2023-09-21 05:47:13.794000+00:00   
                                        354 2023-09-21 06:12:46.061000+00:00   
                                        352 2023-09-21 06:24:40.656000+00:00   
                                        360 2023-09-21 05:59:32.252000+00:00   
...                                                                      ...   
unconditional-moons multivariate_normal 14  2023-09-20 19:48:10.253000+00:00   
                    

### Best validation losses


In [10]:
idx = grouped_df["metrics.val_loss"].idxmin()
df_best_runs = df.loc[idx]
pd.pivot_table(df_best_runs[["experiment_name", "tags.mlflow.runName", "metrics.val_loss"]], values="metrics.val_loss", index="experiment_name", columns=["tags.mlflow.runName"])

tags.mlflow.runName,bernstein_flow,coupling_bernstein_flow,masked_autoregressive_bernstein_flow,multivariate_bernstein_flow,multivariate_normal
experiment_name,,,,,
conditional-circles,-0.744586,-1.595342,-1.553328,-0.742274,-0.174571
conditional-moons,-0.941103,-2.091769,-2.089377,-0.940211,-0.459779
unconditional-circles,-0.131825,-0.738035,-0.634118,-0.133949,0.038944
unconditional-hybrid-circles,NaN,-0.902728,NaN,NaN,NaN
unconditional-hybrid-moons,NaN,-1.286617,NaN,NaN,NaN
unconditional-hybrid-pre-trained-circles,NaN,-0.723111,NaN,NaN,NaN
unconditional-hybrid-pre-trained-moons,NaN,-1.084733,NaN,NaN,NaN
unconditional-moons,-0.110831,-1.244643,-1.346073,-0.201477,0.153429


### Best N by experiment alone

In [11]:
df_best_n = df
df_best_n["rank"] = df_best_n.groupby("experiment_name")["metrics.val_loss"].rank(ascending=True, method="first").replace(np.nan, 1.0).astype(int)

N_BEST_RUNS = 3
df_best_n = df_best_n[df_best_n["rank"] <= N_BEST_RUNS]
df_best_n.pivot_table(values=df_best_n[["run_id","tags.mlflow.runName", "metrics.val_loss"]], index=["experiment_name","rank"], aggfunc="first")

metrics.val_loss  \
experiment_name                          rank                     
conditional-circles                      1            -1.625391   
                                         2            -1.624483   
                                         3            -1.624059   
conditional-moons                        1            -2.118887   
                                         2            -2.118887   
                                         3            -2.116150   
unconditional-circles                    1            -0.738035   
                                         2            -0.738035   
                                         3            -0.738035   
unconditional-hybrid-circles             1            -0.945502   
                                         2            -0.930065   
                                         3            -0.923051   
unconditional-hybrid-moons               1            -1.460489   
                                         2            -1.448670   
                                         3            -1.445103   
unconditional-hybrid-pre-trained-circles 1            -0.822808   
                                         2            -0.819657   
                                         3            -0.818552   
unconditional-hybrid-pre-trained-moons   1            -1.331302   
                                         2            -1.313770   
                                         3            -1.313566   
unconditional-moons                      1            -1.391217   
                                         2            -1.391217   
                                         3            -1.383733   

                                                                         run_id  \
experiment_name                          rank                                     
conditional-circles                      1     a3af65aba3d94abe87edee406d70f420   
                                         2     b12f518b850348f6bdb503f1f1e5e853   
                                         3     25213cf1e8b544eca0e61fb14930f3ed   
conditional-moons                        1     b5f3ae6d2ce44122969e8ca3b4a0496e   
                                         2     55d8cf321e6f4655bc44d4f7d5f1db7f   
                                         3     fd7e6fca9ae4442ba7abd38bd742ac97   
unconditional-circles                    1     87f47ce9d2ef40f0af9d3cc710f90d78   
                                         2     de3bd1edf2a84792b098a8782f7d8273   
                                         3     a67d9faf29c843cf9e208428bf997f77   
unconditional-hybrid-circles             1     258c09f2f3254530a6299d6732b7a5ff   
                                         2     3dc6fa17e2bc4a52b4578c2cd1954150   
                                         3     4fd61cebcaee43c2a620fa19ba35ed26   
unconditional-hybrid-moons               1     3df6600fc61d40539d035aa31c7ac19b   
                                         2     a8beb1fad4754cd79057bae077dfdf51   
                                         3     fca7d7f02f774e5d9ad8509a50a83844   
unconditional-hybrid-pre-trained-circles 1     38007e1610ca41719c93d87a09bf8a50   
                                         2     17eb3bec44eb436897df246441026182   
                                         3     1bc493576bdb4939aa38cb81477e4ef2   
unconditional-hybrid-pre-trained-moons   1     bb271ffb8933497eb1fe620825336123   
                                         2     a57a62129ff54ce6ae248418aa04d2ff   
                                         3     594d9b6b78e549fbad12dc08aeb5ee01   
unconditional-moons                      1     b5963b7aadf94b9dbcbaa06a10230275   
                                         2     561b2bdea373462196b561c287661fd2   
                                         3     aa74a3457b3a4457a7e7aaf96392c493   

                                                                tags.mlflow.runName  
experiment_name                          rank              

### Best N runs for each experiment by runName

In [15]:
df_best_n = df
df_best_n["rank"] = grouped_df["metrics.val_loss"].rank(ascending=True, method="first").replace(np.nan, 1.0).astype(int)

N_BEST_RUNS = 3
df_best_n = df_best_n[df_best_n["rank"] <= N_BEST_RUNS]
df_best_n.pivot_table(values=df_best_n[["run_id", "metrics.val_loss"]], index=["experiment_name", "tags.mlflow.runName","rank"], aggfunc="first")

metrics.val_loss  \
experiment_name     tags.mlflow.runName         rank                     
conditional-circles bernstein_flow              1            -0.744586   
                                                2            -0.744586   
                                                3            -0.742992   
                    coupling_bernstein_flow     1            -1.625391   
                                                2            -1.624483   
...                                                                ...   
unconditional-moons multivariate_bernstein_flow 2            -0.203255   
                                                3            -0.203114   
                    multivariate_normal         1             0.151899   
                                                2             0.153064   
                                                3             0.153268   

                                                                                run_id  
experiment_name     tags.mlflow.runName         rank                                    
conditional-circles bernstein_flow              1     03f38a5a4a784c6ea2381a134d5301d9  
                                                2     7fda6820ed6f4a97bab468c2cd397049  
                                                3     4610459880544b7993a0749702c40623  
                    coupling_bernstein_flow     1     a3af65aba3d94abe87edee406d70f420  
                                                2     b12f518b850348f6bdb503f1f1e5e853  
...                                                                                ...  
unconditional-moons multivariate_bernstein_flow 2     016066ee3b844bfe89ddec3046f95920  
                                                3     587ce76385ba4eb6834b1562e9f58f18  
                    multivariate_normal         1     3a4f6b756e56428a860da00c4bc5e1db  
                                                2     fb194a5d45db4db89b94d0aba421b380  
                                                3     88ba7ff5d6bd491b87da4d7ab8b59b50  

[72 rows x 2 columns]